# TorchServe Model on Vertex AI

## Build Container

In [2]:
%cd torchserve_mnist
!gcloud builds submit --tag gcr.io/sandbox-michael-menzel/torchserve-mnist-example:latest

/home/jupyter/vertex-ai-examples/notebooks/torchserve_mnist
Creating temporary tarball archive of 4 file(s) totalling 4.6 MiB before compression.
Uploading tarball of [.] to [gs://sandbox-michael-menzel_cloudbuild/source/1649339979.561871-656b923e94f343499ce151240b74e799.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/sandbox-michael-menzel/locations/global/builds/e6b18aed-bc57-4a66-aaaa-fe9b5807799e].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/e6b18aed-bc57-4a66-aaaa-fe9b5807799e?project=928871478446].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e6b18aed-bc57-4a66-aaaa-fe9b5807799e"

FETCHSOURCE
Fetching storage object: gs://sandbox-michael-menzel_cloudbuild/source/1649339979.561871-656b923e94f343499ce151240b74e799.tgz#1649339981572550
Copying gs://sandbox-michael-menzel_cloudbuild/source/1649339979.561871-656b923e94f343499ce151240b74e799.tgz#1649339981572550...
/ [1 files][  4.3 MiB/  4.3

## Call Model

In [1]:
!pip install -q --upgrade google-cloud-aiplatform

import base64
import tensorflow as tf
from google.cloud import aiplatform

In [14]:
REGION = "europe-west4"
PROJECT_ID = !gcloud config get-value core/project 2> /dev/null
PROJECT_ID = PROJECT_ID[0]

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
aipe_client = aiplatform.gapic.EndpointServiceClient(client_options=client_options)
aipp_client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    
def get_prediction(content, endpoint_id):
    return aipp_client.predict(endpoint=endpoint_id, instances=[{'data': {'b64': content}}])

def predict(gcs_uri, endpoint_id):
    return get_prediction(base64.b64encode(tf.io.read_file(gcs_uri).numpy()).decode('utf-8'), endpoint_id)

In [15]:
aipe_client.list_endpoints(parent=f"projects/{PROJECT_ID}/locations/{REGION}")

ListEndpointsPager<endpoints {
  name: "projects/808991868713/locations/europe-west4/endpoints/5011626775961141248"
  display_name: "torchserve-mnist-example-ep"
  deployed_models {
    id: "6585265409820196864"
    model: "projects/808991868713/locations/europe-west4/models/1938131136513310720"
    display_name: "torchserve-mnist-example"
    create_time {
      seconds: 1623701789
      nanos: 957949000
    }
    dedicated_resources {
      machine_spec {
        machine_type: "n1-standard-2"
      }
      min_replica_count: 1
      max_replica_count: 1
    }
  }
  traffic_split {
    key: "6585265409820196864"
    value: 100
  }
  etag: "AMEw9yOqsJce1akC8Eg57bk1zfSvQgLxgbiNxZE3GeCbb1B2-5FxN59IMSp4rkxQlizf"
  create_time {
    seconds: 1623701789
    nanos: 957949000
  }
  update_time {
    seconds: 1623702349
    nanos: 458833000
  }
}
>

In [17]:
predict('../serve/examples/image_classifier/mnist/test_data/5.png', 'projects/808991868713/locations/europe-west4/endpoints/5011626775961141248')

predictions {
  number_value: 5.0
}
deployed_model_id: "6585265409820196864"